In [1]:
!pip install antigranular &> /dev/null

In [2]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Global Economic Forecast Hackathon with Texas A&M Aggie Data Science")

Dataset "Statistical Performance Indicators" loaded to the kernel as statistical_performance_indicators
Key Name                       Value Type     
---------------------------------------------
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: 50cca839-4690-4c67-beb0-c40a0fd71a2e, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [3]:
%%ag
x_train = statistical_performance_indicators["train_x"]
y_train = statistical_performance_indicators["train_y"]
x_test = statistical_performance_indicators["test_x"]

In [4]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from op_tensorflow import PrivateKerasModel, PrivateDataLoader
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten,BatchNormalization

model = Sequential([
    Conv1D(256, 5, activation='relu', input_shape=(24, 1)),
    BatchNormalization(),
    Dropout(0.2),
    Conv1D(128, 5, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Conv1D(128, 5, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


dp_model = PrivateKerasModel(model=model, l2_norm_clip=1, noise_multiplier=0.25)

/usr/local/lib/python3.10/site-packages/tensorflow/__init__.py:30: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils as _distutils

2024-12-21 11:50:31.024924: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-21 11:50:31.028061: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-21 11:50:31.139662: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-21 11:50:31.139689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unabl

In [5]:
%%ag
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

In [6]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train , label_df=y_train, batch_size=8)

In [7]:
%%ag
def f(x: float) -> float:
    return 1 if x > 0.5 else 0
x_test=PrivateDataFrame(x_test)


In [ ]:
%%ag
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

In [ ]:
%%ag
dp_model.fit(x=data_loader, epochs=5, target_delta=1)

Epoch 1/5

115/115 - 27s - loss: 0.1474 - accuracy: 0.9451 - 27s/epoch - 235ms/step

Epoch 2/5

115/115 - 27s - loss: 0.1728 - accuracy: 0.9265 - 27s/epoch - 231ms/step

Epoch 3/5

115/115 - 27s - loss: 0.1496 - accuracy: 0.9376 - 27s/epoch - 236ms/step

Epoch 4/5

115/115 - 27s - loss: 0.1520 - accuracy: 0.9411 - 27s/epoch - 235ms/step

Epoch 5/5

115/115 - 27s - loss: 0.1415 - accuracy: 0.9473 - 27s/epoch - 236ms/step


In [ ]:
%%ag
y_pred = dp_model.predict(x_test, label_columns=["output"])
y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))
result = submit_predictions(y_pred)

13/13 [==============================] - 1s 42ms/step

score: {'leaderboard': 0.9653884053828252, 'logs': {'BIN_ACC': 0.9654976555455466, 'LIN_EPS': -0.0001092501627213056}}

